# Imports

All the libraries used in the notebook

In [73]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
import math
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib notebook
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.palettes import Category20
output_notebook()
import plotly.io as pio
pio.renderers.default = "vscode"

Loading BokehJS ...

# Technical Functions

The below functions are the fundemental aspects of this project, they are used to calculate the stabilities for each pertubation.

1. Graph Initialization

In [74]:
def init_graph(G,node_adj_frame):
    G.add_nodes_from([i for i in range(len(node_adj_frame))])
    labels = {}
    labels = node_adj_frame.columns
    for i in range(len(node_adj_frame)):
        snode = node_adj_frame[labels[0]][i]-1
        temp = node_adj_frame[labels[2]][i]
        if ',' in str(temp):
            sedge_arr = temp.split(',')
            for j in range(0, len(sedge_arr)):
                k = int(sedge_arr[j])
                G.add_edge(snode, k-1)
        elif np.isnan(temp):
            print("ERROR: Not found in the adjacency excel sheet")
        else:
            G.add_edge(snode, int(temp)-1)
    return

In [75]:
def init_graph_attr(G, AdjFile, df, columns):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["District_Name"].tolist()
    nodeAttr = {}
    init_graph(G, node_adj_frame)
    
    capability_vector = list(zip(*(df[col] for col in columns)))
    node_attri_dict = dict(zip(df["District"], capability_vector))
    node_attri_dict = dict((k, v) for k, v in node_attri_dict.items())

    for i in range(len(node_adj_frame)):
        temp = {}
        temp["capabilityvector"] = node_attri_dict[node_list[i]]
        temp["nodeStress"] = 0
        temp["nodeStability"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp

    nx.set_node_attributes(G, nodeAttr)

In [76]:
def addList(l1,l2):
    for i in range(len(l1)):
        l1[i] = l1[i] + l2[i]
    return l1

def divList(l1,k):
    for i in range(len(l1)):
        l1[i] = l1[i]/k
    return l1
def l2_normalization(l1,l2):
    k = 0
    for i in range(len(l1)):
        k+= (l1[i] - l2[i])**2
    return math.sqrt(k)

2. Stress Calculation

In [77]:
def get_node_stress(G,dim):
    stress_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return stress_dict

In [78]:
def get_node_stability(G,dim):
    stability_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStability"] = 1 - l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            stability_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStability"]
        except(KeyError):
            pass
    return stability_dict

3. Loading Dataset

In [130]:
G = nx.Graph()
df = pd.read_csv('KAG 2016-17/Agriculture/Agriculture_KAG_2016_17.csv')
adjacency_file = 'Karnataka_District_Adjacency_File - Copy.xlsx'







AttributeError: 'str' object has no attribute 'head'

In [80]:
df.columns

Index(['District', 'NetAreaIrrigated_Canals_Length_171',
       'NetAreaIrrigated_Canals_GrossIrrigatedArea_172',
       'NetAreaIrrigated_Canals_NetAreaIrrigated_173',
       'NetAreaIrrigated_Tanks_No_174',
       'NetAreaIrrigated_Tanks_GrossIrrigatedArea_175',
       'NetAreaIrrigated_Tanks_NetAreaIrrigated_176',
       'NetAreaIrrigated_Wells_No_177',
       'NetAreaIrrigated_Wells_GrossIrrigatedArea_178',
       'NetAreaIrrigated_Wells_NetAreaIrrigated_179',
       ...
       'SowingSeedsDistributed_Safflower_323',
       'SowingSeedsDistributed_Groundnut_328',
       'SowingSeedsDistributed_Sunflower_329',
       'SowingSeedsDistributed_Soyabean_330', 'TotalFoodGrains_240',
       'TotalGourdVarietyVegetables_276', 'RRB_AgricultureLoan_448',
       'DCCBank_AgricultureLoan_466', 'KSCARD_PLDBank_AgricultureLoan_472',
       'TotalAgricultureLoan'],
      dtype='object', length=302)

In [81]:
import re
def remove_pattern(col_name):
    return re.sub(r'_[0-9]{3}$', '', col_name)

df.columns = map(remove_pattern, df.columns)

In [82]:
df = df[['District','TotalNPK', 'Rice_Production','Jowar_Production','Maize_Production','Jowar_Yield']].copy()

In [83]:
df

,District,TotalNPK,Rice_Production,Jowar_Production,Maize_Production,Jowar_Yield
0,BENGALURU,23310,3376,851,2879,1539
1,BENGALURU(R),19259,2354,0,42583,0
2,RAMANAGARA,7472,12128,0,10450,0
3,CHITRADURGA,35884,4566,13301,271975,984
4,DAVANAGERE,99644,489505,23072,593544,2167
5,KOLAR,22190,864,0,1759,0
6,CHIKKABALLAPURA,28357,2698,0,105491,0
7,SHIVAMOGGA,56609,384974,756,226532,2242
8,TUMAKURU,38148,26042,2652,66868,1346
9,CHIKKAMAGALURU,67863,110533,9080,61353,897


4. Running Linear Regression, with base_coloumn as X and Capability_Vector as Y.

Diving Vijayangra Values.

In [84]:
vijayanagara_values = df[df['District'] == 'BALLARI'].iloc[0].copy()
vijayanagara_values['District'] = 'Vijayanagara'
vijayanagara_values[['TotalNPK', 'Rice_Production', 'Jowar_Production', 'Maize_Production', 'Jowar_Yield']] = (vijayanagara_values[['TotalNPK', 'Rice_Production', 'Jowar_Production', 'Maize_Production', 'Jowar_Yield']].astype(float) / 2).round().astype(int)
df = pd.concat([df, pd.DataFrame([vijayanagara_values], columns=df.columns)], ignore_index=True)

df.loc[df['District'] == 'BALLARI', ['TotalNPK', 'Rice_Production', 'Jowar_Production', 'Maize_Production', 'Jowar_Yield']] /= 2
df.loc[df['District'] == 'BALLARI', ['TotalNPK', 'Rice_Production', 'Jowar_Production', 'Maize_Production', 'Jowar_Yield']] = df.loc[df['District'] == 'BALLARI', ['TotalNPK', 'Rice_Production', 'Jowar_Production', 'Maize_Production', 'Jowar_Yield']].applymap(lambda x: x + 0.5 if x % 1 != 0 else x)

C:\Users\dalal\AppData\Local\Temp\ipykernel_12796\1542201205.py:6: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[61664.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\dalal\AppData\Local\Temp\ipykernel_12796\1542201205.py:6: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[206115.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\dalal\AppData\Local\Temp\ipykernel_12796\1542201205.py:6: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[903.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\dalal\AppData\Local\Temp\ipykernel_12796\1542201205.py:7: FutureWarning:

DataFrame.applymap has been de

In [85]:
base_column = "TotalNPK"
capability_vector = "Rice_Production"
X = sm.add_constant(df[base_column])
y = df[capability_vector]
model = sm.OLS(y, X).fit()
m, c = model.params[base_column], model.params['const']
print(m,c)

2.1764894064966205 17136.471837471167


In [86]:
def calculate_and_return_initial_stability(df, columns, G, adjacency_file = adjacency_file):
    dim = len(columns)
    col_to_pass = []

    for column in columns:
        column_values = df[column].values.reshape(-1, 1)
        scaler = MinMaxScaler()
        normalized_column_values = scaler.fit_transform(column_values)
        df[f"Normalized_{column}"] = normalized_column_values
        col_to_pass.append(f"Normalized_{column}")

    init_graph_attr(G, adjacency_file, df, col_to_pass)
    initial_stability = get_node_stability(G, dim)
    df["Initial Stability"] = df["District"].map(initial_stability)

    return df["Initial Stability"]

In [87]:
df

,District,TotalNPK,Rice_Production,Jowar_Production,Maize_Production,Jowar_Yield
0,BENGALURU,23310.0,3376.0,851,2879,1539.0
1,BENGALURU(R),19259.0,2354.0,0,42583,0.0
2,RAMANAGARA,7472.0,12128.0,0,10450,0.0
3,CHITRADURGA,35884.0,4566.0,13301,271975,984.0
4,DAVANAGERE,99644.0,489505.0,23072,593544,2167.0
5,KOLAR,22190.0,864.0,0,1759,0.0
6,CHIKKABALLAPURA,28357.0,2698.0,0,105491,0.0
7,SHIVAMOGGA,56609.0,384974.0,756,226532,2242.0
8,TUMAKURU,38148.0,26042.0,2652,66868,1346.0
9,CHIKKAMAGALURU,67863.0,110533.0,9080,61353,897.0


Normalizing Production Values 

In [88]:
columns_to_normalize = ["Maize_Production","Rice_Production","Jowar_Production"]
dim = 1
for col in columns_to_normalize:
    column_values = df[col].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    normalized_values = scaler.fit_transform(column_values)
    df[f"Normalized_{col}"] = normalized_values.flatten()
    init_graph_attr(G,adjacency_file , df,[f"Normalized_{col}"] )
    initial_stability = get_node_stability(G, dim)
    df[f"Initial Stability_{col}"] = df["District"].map(initial_stability)

In [89]:
df

,District,TotalNPK,Rice_Production,Jowar_Production,Maize_Production,Jowar_Yield,Normalized_Maize_Production,Initial Stability_Maize_Production,Normalized_Rice_Production,Initial Stability_Rice_Production,Normalized_Jowar_Production,Initial Stability_Jowar_Production
0,BENGALURU,23310.0,3376.0,851,2879,1539.0,0.004851,0.960176,0.006686,0.992148,0.003267,0.996733
1,BENGALURU(R),19259.0,2354.0,0,42583,0.0,0.071744,0.991418,0.004609,0.986455,0.000000,0.997311
2,RAMANAGARA,7472.0,12128.0,0,10450,0.0,0.017606,0.923391,0.024465,0.892930,0.000000,0.992597
3,CHITRADURGA,35884.0,4566.0,13301,271975,984.0,0.458222,0.855101,0.009103,0.587418,0.051060,0.972188
4,DAVANAGERE,99644.0,489505.0,23072,593544,2167.0,1.000000,0.331747,0.994237,0.356941,0.088568,0.993075
5,KOLAR,22190.0,864.0,0,1759,0.0,0.002964,0.878226,0.001583,0.996624,0.000000,1.000000
6,CHIKKABALLAPURA,28357.0,2698.0,0,105491,0.0,0.177731,0.884725,0.005308,0.985667,0.000000,0.996607
7,SHIVAMOGGA,56609.0,384974.0,756,226532,2242.0,0.381660,0.986495,0.781886,0.630976,0.002902,0.950160
8,TUMAKURU,38148.0,26042.0,2652,66868,1346.0,0.112659,0.909238,0.052731,0.901262,0.010180,0.996715
9,CHIKKAMAGALURU,67863.0,110533.0,9080,61353,897.0,0.103367,0.744033,0.224371,0.841603,0.034856,0.987498


In [90]:
existing_data = pd.DataFrame(df['District'])

In [91]:
dim = 1

In [92]:
def calculate_and_map_stability(G, existing_data, adjacency_file, columns_to_pass, inter, dim):
    # Initialize the graph attributes
    init_graph_attr(G, adjacency_file, existing_data, columns_to_pass)

    # Calculate node stress for the given change percentage
    NPK_stability = get_node_stability(G, dim)

    # Create a new column in the result DataFrame
    stability_column_name = f"New Stability(NPK {'+' if inter >= 0 else '-'} {abs(inter)}%)"
    stability_column = existing_data["District"].map(NPK_stability)

    return stability_column, stability_column_name

In [93]:
def calculate_and_map_stress(G, existing_data, adjacency_file, columns_to_pass, inter, dim):
    # Initialize the graph attributes
    init_graph_attr(G, adjacency_file, existing_data, columns_to_pass)

    # Calculate node stress for the given change percentage
    NPK_stress = get_node_stress(G, dim)

    # Create a new column in the result DataFrame
    stability_column_name = f"New Stability(NPK {'+' if inter >= 0 else '-'} {abs(inter)}%)"
    stability_column = existing_data["District"].map(NPK_stress)

    return stability_column, stability_column_name

## 1D Capability Vector

In [127]:
df
Sparsh = pd.DataFrame()
Sparsh['District'] = df['District']
Sparsh['Normalized_Rice_Production'] = df['Normalized_Rice_Production']
Sparsh.to_csv('Normal_Rice.csv')

In [95]:
from bokeh.models import Span, Label
from bokeh.plotting import figure, show, output_notebook
from bokeh.models.sources import ColumnDataSource
from bokeh.models.tools import HoverTool
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px

def calc_and_vis_impact_stability_1D(G, existing_data, adjacency_file, base_column, capability_vector, intervention, dim, final_data):
    filtered_df = df[(df[capability_vector] != 0) & (~df[capability_vector].isnull())]
    X = sm.add_constant(df[base_column])
    y = df[capability_vector]
    model = sm.OLS(y, X).fit()
    
    m, c = model.params[base_column], model.params['const']
    print(m,c)
    base_column_increase = (1 + intervention/100) * df[base_column]
    base_column_decrease = (1 - intervention/100) * df[base_column]

    new_vector_increase = m * base_column_increase + c
    new_vector_decrease = m * base_column_decrease + c
    
    vector_plus = new_vector_increase - m * df[base_column] - c  
    vector_minus = new_vector_decrease - m * df[base_column] - c

    scaler = MinMaxScaler()
    normalized_change_increase_vector = scaler.fit_transform(vector_plus.values.reshape(-1, 1))
    normalized_change_decrease_vector = scaler.fit_transform(vector_minus.values.reshape(-1, 1))
    normalized_new_increase_vector = scaler.fit_transform(new_vector_increase.values.reshape(-1, 1))
    normalized_new_decrease_vector = scaler.fit_transform(new_vector_decrease.values.reshape(-1, 1))

    result_df = pd.DataFrame({
        f'{capability_vector} ({base_column} +{intervention}%)': new_vector_increase,
        f'{capability_vector} ({base_column} -{intervention}%)': new_vector_decrease,
        f'Normalized {capability_vector} ({base_column} +{intervention}%)': normalized_new_increase_vector.flatten(),
        f'Normalized {capability_vector} ({base_column} -{intervention}%)': normalized_new_decrease_vector.flatten(),
        f'Impact Score {capability_vector} ({base_column} +{intervention}%)': normalized_change_increase_vector.flatten(),
        f'Impact Score {capability_vector} ({base_column} -{intervention}%)': normalized_change_decrease_vector.flatten(),
    })
    
    # Merge Columns
    for column in result_df.columns:
        existing_data[column] = result_df[column]

    # Calculate Stability and add to DataFrame
    columns_to_pass = [f"Normalized {capability_vector} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"]
    stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass, intervention, dim)

    existing_data[stability_column_name] = stability_column

    # Calculate Stability for the opposite intervention and add to DataFrame
    opposite_intervention = -intervention
    columns_to_pass_opposite = [f"Normalized {capability_vector} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"]
    stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_opposite, opposite_intervention, dim)

    existing_data[stability_column_name_opposite] = stability_column_opposite

    columns_to_pass = [f"Normalized {capability_vector} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"]
    stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass, intervention, dim)

    existing_data[stability_column_name] = stability_column

    # Calculate Stability for the opposite intervention and add to DataFrame
    opposite_intervention = -intervention
    columns_to_pass_opposite = [f"Normalized {capability_vector} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"]
    stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_opposite, opposite_intervention, dim)

    existing_data[stability_column_name_opposite] = stability_column_opposite

    # Visualize with Plotly for +intervention
    fig_increase = px.scatter(existing_data,
                              x=stability_column_name,
                              y=f'Impact Score {capability_vector} ({base_column} +{intervention}%)',
                              title=f'Impact vs Stability ({capability_vector} - {base_column} +{intervention}%)',
                              labels={'x': f'Stability ({base_column} +{intervention}%)', 'y': f'Impact ({capability_vector})'},
                              size_max=8,
                              width=800,
                              height=500)
    
    fig_increase.update_traces(hoverlabel=dict(bgcolor='grey', font=dict(color='white')))

    # Calculate average values from the DataFrame for +intervention
    avg_x_increase = existing_data[stability_column_name].mean()
    avg_y_increase = existing_data[f'Normalized {capability_vector} ({base_column} +{intervention}%)'].mean()

    # Add average lines for +intervention
    fig_increase.add_shape(
        type='line',
        x0=avg_x_increase,
        x1=avg_x_increase,
        y0=fig_increase.data[0].y.min(),
        y1=fig_increase.data[0].y.max(),
        line=dict(color='red', width=2)
    )
    fig_increase.add_shape(
        type='line',
        x0=fig_increase.data[0].x.min(),
        x1=fig_increase.data[0].x.max(),
        y0=avg_y_increase,
        y1=avg_y_increase,
        line=dict(color='blue', width=2)
    )

    # Add labels for average lines for +intervention
    fig_increase.add_annotation(
        x=avg_x_increase,
        y=fig_increase.data[0].y.max(),
        text=f'Avg {stability_column_name}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='red',
        ax=0,
        ay=-40,
        font=dict(color='red')
    )
    fig_increase.add_annotation(
        x=fig_increase.data[0].x.max(),
        y=avg_y_increase,
        text=f'Avg Normalized {capability_vector}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='blue',
        ax=-40,
        ay=0,
        font=dict(color='blue')
    )

    # Add tooltips for +intervention
    fig_increase.update_traces(hoverinfo='text+name',
                               hovertext=["District: " + str(d) +
                                          f"<br>Impact ({capability_vector}): {y} <br>Stability ({base_column} +{intervention}%): {x}"
                                          for x, y, d in zip(fig_increase.data[0].x, fig_increase.data[0].y, existing_data['District'])])

    # Visualize with Plotly for -intervention
    fig_decrease = px.scatter(existing_data,
                              x=stability_column_name_opposite,
                              y=f'Impact Score {capability_vector} ({base_column} -{intervention}%)',
                              title=f'Impact vs Stability ({capability_vector} - {base_column} -{intervention}%)',
                              labels={'x': f'Stability ({base_column} -{intervention}%)', 'y': f'Impact ({capability_vector})'},
                              size_max=8,
                              width=800,
                              height=500)

    # Calculate average values from the DataFrame for -intervention
    avg_x_decrease = existing_data[stability_column_name_opposite].mean()
    avg_y_decrease = existing_data[f'Normalized {capability_vector} ({base_column} -{intervention}%)'].mean()

    # Add average lines for -intervention
    fig_decrease.add_shape(
        type='line',
        x0=avg_x_decrease,
        x1=avg_x_decrease,
        y0=fig_decrease.data[0].y.min(),
        y1=fig_decrease.data[0].y.max(),
        line=dict(color='red', width=2)
    )
    fig_decrease.add_shape(
        type='line',
        x0=fig_decrease.data[0].x.min(),
        x1=fig_decrease.data[0].x.max(),
        y0=avg_y_decrease,
        y1=avg_y_decrease,
        line=dict(color='blue', width=2)
    )

    # Add labels for average lines for -intervention
    fig_decrease.add_annotation(
        x=avg_x_decrease,
        y=fig_decrease.data[0].y.max(),
        text=f'Avg {stability_column_name_opposite}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='red',
        ax=0,
        ay=-40,
        font=dict(color='red')
    )
    fig_decrease.add_annotation(
        x=fig_decrease.data[0].x.max(),
        y=avg_y_decrease,
        text=f'Avg Normalized {capability_vector}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='blue',
        ax=-40,
        ay=0,
        font=dict(color='blue')
    )

    # Add tooltips for -intervention
    fig_decrease.update_traces(hoverinfo='text+name',
                               hovertext=["District: " + str(d) +
                                          f"<br>Impact ({capability_vector}): {y} <br>Stability ({base_column} -{intervention}%): {x}"
                                          for x, y, d in zip(fig_decrease.data[0].x, fig_decrease.data[0].y, existing_data['District'])])

    # Show the plots
    fig_increase.show()
    fig_decrease.show()

    return existing_data

base_Column = "TotalNPK"
CapabilityVector = "Rice_Production"
adjacency_file = 'Karnataka_District_Adjacency_File - Copy.xlsx'
change_percentage = 20
inter = [10,20]
dim = 1

# Call the function and get the handles for the two plots
new_data_20 = calc_and_vis_impact_stability_1D(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# Display the resulting DataFrame
new_data_20.head()


2.1764894064966205 17136.471837471167


,District,Rice_Production (TotalNPK +20%),Rice_Production (TotalNPK -20%),Normalized Rice_Production (TotalNPK +20%),Normalized Rice_Production (TotalNPK -20%),Impact Score Rice_Production (TotalNPK +20%),Impact Score Rice_Production (TotalNPK -20%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,BENGALURU,78017.233516,57723.646290,0.098721,0.098721,0.098721,0.901279,0.942796,0.942796
1,BENGALURU(R),67436.883213,50670.079421,0.075418,0.075418,0.075418,0.924582,0.973330,0.973330
2,RAMANAGARA,36651.746452,30146.654914,0.007616,0.007616,0.007616,0.992384,0.853066,0.853066
3,CHITRADURGA,110857.846873,79617.388528,0.171050,0.171050,0.171050,0.828950,0.827933,0.827933
4,DAVANAGERE,277385.404343,190635.760174,0.537816,0.537816,0.537816,0.462184,0.761110,0.761110


In [96]:
new_data_20

,District,Rice_Production (TotalNPK +20%),Rice_Production (TotalNPK -20%),Normalized Rice_Production (TotalNPK +20%),Normalized Rice_Production (TotalNPK -20%),Impact Score Rice_Production (TotalNPK +20%),Impact Score Rice_Production (TotalNPK -20%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,BENGALURU,78017.233516,57723.646290,0.098721,0.098721,0.098721,0.901279,0.942796,0.942796
1,BENGALURU(R),67436.883213,50670.079421,0.075418,0.075418,0.075418,0.924582,0.973330,0.973330
2,RAMANAGARA,36651.746452,30146.654914,0.007616,0.007616,0.007616,0.992384,0.853066,0.853066
3,CHITRADURGA,110857.846873,79617.388528,0.171050,0.171050,0.171050,0.828950,0.827933,0.827933
4,DAVANAGERE,277385.404343,190635.760174,0.537816,0.537816,0.537816,0.462184,0.761110,0.761110
5,KOLAR,75092.031754,55773.511782,0.092278,0.092278,0.092278,0.907722,0.990693,0.990693
6,CHIKKABALLAPURA,91198.923958,66511.439917,0.127752,0.127752,0.127752,0.872248,0.989504,0.989504
7,SHIVAMOGGA,164987.138412,115703.582887,0.290266,0.290266,0.290266,0.709734,0.966553,0.966553
8,TUMAKURU,116770.933292,83559.446141,0.184073,0.184073,0.184073,0.815927,0.982917,0.982917
9,CHIKKAMAGALURU,194380.192549,135298.952312,0.355002,0.355002,0.355002,0.644998,0.875431,0.875431


In [97]:
change_percentage = 10
dim = 1

# Call the function and get the handles for the two plots
new_data = calc_and_vis_impact_stability_1D(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# Display the resulting DataFrame
new_data.head()

2.1764894064966205 17136.471837471167


,District,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Impact Score Rice_Production (TotalNPK +10%),Impact Score Rice_Production (TotalNPK -10%),New Stability(NPK + 10%),New Stability(NPK - 10%)
0,BENGALURU,72943.836709,62797.043096,0.098721,0.098721,0.098721,0.901279,0.942796,0.942796
1,BENGALURU(R),63245.182265,54861.780369,0.075418,0.075418,0.075418,0.924582,0.973330,0.973330
2,RAMANAGARA,35025.473567,31772.927798,0.007616,0.007616,0.007616,0.992384,0.853066,0.853066
3,CHITRADURGA,103047.732286,87427.503114,0.171050,0.171050,0.171050,0.828950,0.827933,0.827933
4,DAVANAGERE,255697.993301,212323.171216,0.537816,0.537816,0.537816,0.462184,0.761110,0.761110


## 2D Capability vector

In [98]:
res_df = pd.DataFrame()

In [99]:
import statsmodels.api as sm
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import cycle

def calc_and_vis_impact_stability_2D1(G, existing_data, adjacency_file, base_column, capability_vector, change_percentages, dim):
    change_min_value = float('inf')
    change_max_value = float('-inf')
    
    initial_stability_column = calculate_and_return_initial_stability(df, capability_vector, G)
    existing_data = pd.concat([existing_data, initial_stability_column], axis=1)

    for intervention in change_percentages:
        for cap_col in capability_vector:
            # filtered_df = df[(df[cap_col] != 0) & (~df[cap_col].isnull())]
            X = sm.add_constant(df[base_column])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()
            m, c = model.params[base_column], model.params['const']

            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            change_vector_increase = m * base_column_increase + c
            change_vector_decrease = m * base_column_decrease + c
            
            change_min_value = min(change_min_value, change_vector_increase.min(), change_vector_decrease.min())
            change_max_value = max(change_max_value, change_vector_increase.max(), change_vector_decrease.max())
            
    
    result_df = pd.DataFrame()
    res1_df = pd.DataFrame()
    
    for intervention in change_percentages:
        col_pass_plus = []
        col_pass_minus = []
        for cap_col in capability_vector:
            # Calculate Impact Score
            filtered_df = df[(df[cap_col] != 0) & (~df[cap_col].isnull())]
            X = sm.add_constant(filtered_df[base_column])
            y = filtered_df[cap_col]
            # X = sm.add_constant(df[base_column])
            # y = df[cap_col]
            model = sm.OLS(y, X).fit()

            m, c = model.params[base_column], model.params['const']
            # print(X[0,:])
            # print(y)
            print(m,c,cap_col,intervention)
            # print(m,c)
            
            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            new_vector_increase = m * base_column_increase + c
            new_vector_decrease = m * base_column_decrease + c
            
            if any(y == 0):
                new_vector_increase[y == 0] = 0
                new_vector_decrease[y == 0] = 0
                
            normalized_new_increase_vector = (new_vector_increase - change_min_value) / (change_max_value - change_min_value)
            normalized_new_decrease_vector = (new_vector_decrease - change_min_value) / (change_max_value - change_min_value)
            
            percent_change_inc = (new_vector_increase - (y) )*100/(y + 1e-9)
            percent_change_dec = (new_vector_decrease - (y) )*100/(y + 1e-9)
            
            result_df = pd.concat([result_df,
                pd.DataFrame({
                    f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
                    f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
                    f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector,
                    f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector,
                    f'Percent Change in {cap_col} ({base_column} +{intervention}%)': percent_change_inc,
                    f'Percent Change in {cap_col} ({base_column} -{intervention}%)': percent_change_dec,
                })
            ], axis=1)

            columns_to_pass = f'{cap_col} ({base_column} +{intervention}%)'
            col_pass_plus.append(columns_to_pass)
            
            # Calculate Stability for the opposite intervention and add to DataFrame
            opposite_intervention = -intervention
            columns_to_pass_opposite = f'{cap_col} ({base_column} -{intervention}%)'
            col_pass_minus.append(columns_to_pass_opposite)
        
        for column in result_df.columns:
            existing_data[column] = result_df[column]

        # print(col_pass_minus)
        # print(col_pass_plus)
        stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

        # normalized_stability_vector = (stability_column - stability_column.min()) / (stability_column.max() - stability_column.min())

        capability_first_words = [col.split('_')[0] for col in capability_vector]
        stability_column_name_modified = stability_column_name + "_" + "_".join(capability_first_words)
        # existing_data[stability_column_name_modified] = normalized_stability_vector
        
        stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)

        # Normalize stability values for the opposite intervention
        # normalized_stability_vector_opposite = (stability_column_opposite - stability_column_opposite.min()) / (stability_column_opposite.max() - stability_column_opposite.min())    
        stability_column_name_opposite_modified = stability_column_name_opposite + "_" + "_".join(capability_first_words)
        # existing_data[stability_column_name_opposite_modified] = normalized_stability_vector_opposite

###################
        stress_column, stress_column_name = calculate_and_map_stress(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)
        capability_first_words = [col.split('_')[0] for col in capability_vector]
        stress_column_name_modified = stress_column_name + "_" + "_".join(capability_first_words)        
        stress_column_opposite, stress_column_name_opposite = calculate_and_map_stress(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)
        stress_column_name_opposite_modified = stress_column_name_opposite + "_" + "_".join(capability_first_words)
##############    
        res1_df = pd.concat([res1_df,
                pd.DataFrame({
                    stability_column_name_modified: stress_column,
                    stability_column_name_opposite_modified: stress_column_opposite,
                })
            ], axis=1)
        
    normalized_data_1 = res1_df.copy()
    min_value = normalized_data_1.min().min()
    max_value = normalized_data_1.max().max()
    normalized_data_1 = 1 - (normalized_data_1 - min_value) / (max_value - min_value)
    
    for column in normalized_data_1.columns:
        existing_data[column] = normalized_data_1[column]
        res1_df[column] = normalized_data_1[column]
        
        
    color_palette = ['red', 'blue', 'green', 'orange', 'purple', 'violet', 'yellow']
    color_cycle = cycle(color_palette)
    capability_color_map = {cap_col: next(color_cycle) for cap_col in capability_vector}
    fig = make_subplots(rows=len(change_percentages), cols=2, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.1, horizontal_spacing=0.1, row_heights=[0.5] * len(change_percentages))
    
    for idx, intervention in enumerate(change_percentages):
        for cap_col in capability_vector:
            color = capability_color_map[cap_col]

            fig.update_xaxes(title_text=f'Stability ({base_column} +{intervention}%)', row=idx + 1, col=1, showline=True, showgrid=True)
            fig.update_yaxes(title_text='Percent Change', row=idx + 1, col=1, showline=True, showgrid=True)
            fig.update_xaxes(title_text=f'Stability ({base_column} -{intervention}%)', row=idx + 1, col=2, showline=True, showgrid=True)
            fig.update_yaxes(title_text='Percent Change', row=idx + 3, col=2, showline=True, showgrid=True)
            
            fig.update_layout(margin=dict(t=50))
            
            scatter_increase = go.Scatter(
                x=existing_data[stability_column_name_modified],
                y=existing_data[f"Percent Change in {cap_col} ({base_column} +{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=5, symbol='circle', color=color),
                name=f'{cap_col} +{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Percent Change: %{y:.2f}' + '<br>Stability: %{x:.2f}',
                showlegend=False
            )

            fig.add_trace(scatter_increase, row=idx + 1, col=1)

            scatter_decrease = go.Scatter(
                x=existing_data[stability_column_name_opposite_modified],
                y=existing_data[
                    f"Percent Change in {cap_col} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=5, symbol='circle', color=color),
                name=f'{cap_col} -{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Percent Change: %{y:.2f}' + '<br>Stability: %{x:.2f}',
                showlegend=False
            )

            fig.add_trace(scatter_decrease, row=idx + 1, col=2)

        avg_stability_increase = existing_data[stability_column_name_modified].mean()
        avg_stability_decrease = existing_data[stability_column_name_opposite_modified].mean()
        all_values = [
            existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].min(),
            existing_data[f"Percent Change in {capability_vector[0]} ({base_column} -{abs(intervention)}%)"].min(),
            existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].max(),
            existing_data[f"Percent Change in {capability_vector[0]} ({base_column} -{abs(intervention)}%)"].max()
        ]

        filtered_values = [value for value in all_values if not np.isinf(value)]

        # Calculate the range
        if filtered_values:
            y_range_1 = max(abs(min(filtered_values)), max(filtered_values))
        else:
            y_range_1 = 0  # If all values are inf or -inf, set the range to 0

        y_axis_1 = max(y_range_1, 1)
        print(y_axis_1)
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_increase,
                x1=avg_stability_increase,
                y0=-10000,
                y1=y_axis_1,
                line=dict(color="black", dash="dash", width=1),
                name="Avg Stability"
            ),
            row=idx + 1, col=1
        )
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_decrease,
                x1=avg_stability_decrease,
                y0=-10000,
                y1=y_axis_1,
                line=dict(color="black", dash="dash", width=1),
                name="Avg Stability"
            ),
            row=idx + 1, col=2
        )

        avg_impact_increase = existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].mean()
        avg_impact_decrease = existing_data[f"Percent Change in {capability_vector[0]} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"].mean()

        # Add shapes for average impact lines
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_increase,
                y1=avg_impact_increase,
                line=dict(color="black", dash="dash", width=1),
                name="Avg Impact"
            ),
            row=idx + 1, col=1
        )
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_decrease,
                y1=avg_impact_decrease,
                line=dict(color="black", dash="dash", width=1),
                name="Avg Impact"
            ),
            row=idx + 1, col=2
        )


    for cap_col in capability_vector:
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(size=5, symbol='circle', color=capability_color_map[cap_col]), name=cap_col), row=1, col=1)

    fig.update_layout(
        height=800,  
        showlegend=True,
        legend=dict(title="Capability", orientation="v", yanchor="bottom", y=0.85, xanchor="left", x=1),
    )
    fig.show()

    return existing_data, result_df, res1_df

base_column = "TotalNPK"
capability_vector = ["Rice_Production", "Maize_Production"]
change_percentages = [10,20]
dim = len(capability_vector)

new_data_1,result_df,res1_df = calc_and_vis_impact_stability_2D1(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
new_data_1.head()

2.1764894064966205 17136.471837471167 Rice_Production 10
2.6936229082208722 -4743.442857301005 Maize_Production 10
2.1764894064966205 17136.471837471167 Rice_Production 20
2.6936229082208722 -4743.442857301005 Maize_Production 20
258944.86274476734
280661.6179849578


,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 10%)_Rice_Maize,New Stability(NPK - 10%)_Rice_Maize,New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.959409,72943.836709,62797.043096,0.117432,0.099835,2060.658670,1760.101987,64323.742132,51766.072134,...,70602.577131,45487.237135,0.113372,0.069814,2352.329876,1479.966556,0.940864,0.953059,0.934767,0.959157
1,BENGALURU(R),0.983965,63245.182265,54861.780369,0.100612,0.086073,2586.711226,2230.576906,52320.689091,41945.392373,...,57508.337450,36757.744014,0.090663,0.054675,35.049990,-13.679769,0.976666,0.982351,0.973823,0.985194
2,RAMANAGARA,0.868345,35025.473567,31772.927798,0.051671,0.046030,188.798430,161.979946,17395.982550,13370.632476,...,19408.657587,11357.957439,0.024586,0.010624,85.728781,8.688588,0.835654,0.866978,0.819992,0.882640
3,CHITRADURGA,0.562713,103047.732286,87427.503114,0.169641,0.142551,2156.849152,1814.750397,101580.318025,82248.725137,...,111246.114469,72582.928694,0.183860,0.116806,-59.096934,-73.312647,0.806185,0.842867,0.787844,0.861208
4,DAVANAGERE,0.072591,255697.993301,212323.171216,0.434382,0.359157,-47.763967,-56.624923,290500.254316,236819.582103,...,317340.590423,209979.245996,0.541288,0.355092,-46.534614,-64.622800,0.727833,0.778761,0.702369,0.804225


In [124]:
new_data_1.columns

Index(['District', 'Initial Stability', 'Rice_Production (TotalNPK +10%)',
       'Rice_Production (TotalNPK -10%)',
       'Normalized Rice_Production (TotalNPK +10%)',
       'Normalized Rice_Production (TotalNPK -10%)',
       'Percent Change in Rice_Production (TotalNPK +10%)',
       'Percent Change in Rice_Production (TotalNPK -10%)',
       'Maize_Production (TotalNPK +10%)', 'Maize_Production (TotalNPK -10%)',
       'Normalized Maize_Production (TotalNPK +10%)',
       'Normalized Maize_Production (TotalNPK -10%)',
       'Percent Change in Maize_Production (TotalNPK +10%)',
       'Percent Change in Maize_Production (TotalNPK -10%)',
       'Rice_Production (TotalNPK +20%)', 'Rice_Production (TotalNPK -20%)',
       'Normalized Rice_Production (TotalNPK +20%)',
       'Normalized Rice_Production (TotalNPK -20%)',
       'Percent Change in Rice_Production (TotalNPK +20%)',
       'Percent Change in Rice_Production (TotalNPK -20%)',
       'Maize_Production (TotalNPK +20%)', 'M

In [101]:
new_data_1['Percent Change in Rice_Production (TotalNPK +10%)']

0       2060.658670
1       2586.711226
2        188.798430
3       2156.849152
4        -47.763967
5       8032.222426
6       3051.484542
7        -60.343751
8        316.512025
9         62.494352
10       -53.511090
11       -76.942272
12        25.931424
13        33.447568
14       -34.242423
15       -53.248463
16        61.645470
17       152.355491
18     86350.474192
19    258944.862745
20       213.724315
21       904.023320
22        37.741272
23       -64.828904
24       -20.059087
25      3641.214892
26      1756.206243
27         5.018887
28       -39.510893
29       -20.646338
30       -20.060248
Name: Percent Change in Rice_Production (TotalNPK +10%), dtype: float64

In [102]:
new_data_1['Percent Change in Rice_Production (TotalNPK -10%)']

0       1760.101987
1       2230.576906
2        161.979946
3       1814.750397
4        -56.624923
5       6914.252520
6       2593.970012
7        -66.744645
8        252.746786
9         35.768741
10       -59.613692
11       -78.879366
12         5.202939
13        12.794587
14       -44.934582
15       -60.869159
16        39.049964
17       108.227497
18     72464.983893
19    215511.352264
20       165.850385
21       764.401734
22        15.181596
23       -69.186976
24       -33.082163
25      3114.025542
26      1458.213719
27       -12.688589
28       -49.876075
29       -33.794745
30       -33.083113
Name: Percent Change in Rice_Production (TotalNPK -10%), dtype: float64

In [103]:
abbreviation_mapping = {
    'BENGALURU': 'BLR',
    'BENGALURU(R)': 'BLR(R)',
    'RAMANAGARA': 'RGA',
    'CHITRADURGA': 'CDA',
    'DAVANAGERE': 'DVG',
    'KOLAR': 'KLR',
    'CHIKKABALLAPURA': 'CKA',
    'SHIVAMOGGA': 'SMG',
    'TUMAKURU': 'TKR',
    'CHIKKAMAGALURU': 'CMG',
    'DAKSHINA KANNADA': 'DKA',
    'UDUPI': 'UPI',
    'HASSAN': 'HSN',
    'KODAGU': 'KDG',
    'MANDYA': 'MDY',
    'MYSURU': 'MYS',
    'CHAMARAJANAGAR': 'CNR',
    'BELAGAVI': 'BLG',
    'VIJAYAPURA': 'VJP',
    'BAGALKOT': 'BKT',
    'DHARAWAD': 'DWD',
    'GADAG': 'GDG',
    'HAVERI': 'HVR',
    'UTTARA KANNADA': 'UTK',
    'BALLARI': 'BLL',
    'BIDAR': 'BDR',
    'KALABURAGI': 'KLB',
    'YADGIRI': 'YDR',
    'RAICHUR': 'RCR',
    'KOPPAL': 'KPL',
    'Vijayanagara': 'VNG',
}

df['Abbreviation'] = df['District'].map(abbreviation_mapping)

In [104]:
df['Abbreviation']

0        BLR
1     BLR(R)
2        RGA
3        CDA
4        DVG
5        KLR
6        CKA
7        SMG
8        TKR
9        CMG
10       DKA
11       UPI
12       HSN
13       KDG
14       MDY
15       MYS
16       CNR
17       BLG
18       VJP
19       BKT
20       DWD
21       GDG
22       HVR
23       UTK
24       BLL
25       BDR
26       KLB
27       YDR
28       RCR
29       KPL
30       VNG
Name: Abbreviation, dtype: object

In [105]:
res1_df.columns = [col.replace('_Rice_Maize', '') for col in res1_df.columns]

print(res1_df.columns)

Index(['New Stability(NPK + 10%)', 'New Stability(NPK - 10%)',
       'New Stability(NPK + 20%)', 'New Stability(NPK - 20%)'],
      dtype='object')


In [106]:
res1_df

,New Stability(NPK + 10%),New Stability(NPK - 10%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,0.940864,0.953059,0.934767,0.959157
1,0.976666,0.982351,0.973823,0.985194
2,0.835654,0.866978,0.819992,0.882640
3,0.806185,0.842867,0.787844,0.861208
4,0.727833,0.778761,0.702369,0.804225
5,0.997024,0.999008,0.996032,1.000000
6,0.995630,0.997867,0.994511,0.998986
7,0.968719,0.975850,0.965154,0.979415
8,0.987907,0.991549,0.986086,0.993370
9,0.861877,0.888433,0.848599,0.901712


In [107]:
new_data_1.columns

Index(['District', 'Initial Stability', 'Rice_Production (TotalNPK +10%)',
       'Rice_Production (TotalNPK -10%)',
       'Normalized Rice_Production (TotalNPK +10%)',
       'Normalized Rice_Production (TotalNPK -10%)',
       'Percent Change in Rice_Production (TotalNPK +10%)',
       'Percent Change in Rice_Production (TotalNPK -10%)',
       'Maize_Production (TotalNPK +10%)', 'Maize_Production (TotalNPK -10%)',
       'Normalized Maize_Production (TotalNPK +10%)',
       'Normalized Maize_Production (TotalNPK -10%)',
       'Percent Change in Maize_Production (TotalNPK +10%)',
       'Percent Change in Maize_Production (TotalNPK -10%)',
       'Rice_Production (TotalNPK +20%)', 'Rice_Production (TotalNPK -20%)',
       'Normalized Rice_Production (TotalNPK +20%)',
       'Normalized Rice_Production (TotalNPK -20%)',
       'Percent Change in Rice_Production (TotalNPK +20%)',
       'Percent Change in Rice_Production (TotalNPK -20%)',
       'Maize_Production (TotalNPK +20%)', 'M

In [108]:
ch = [-10, 10, 20, -20]

for intervention in ch:
    for cap_col in capability_vector:
        percent_change_col = new_data_1[f"Percent Change in {cap_col} (TotalNPK {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"]
        percent_change_col.replace([np.inf, -np.inf], np.nan, inplace=True)
    
        min_val = percent_change_col.min()
        max_val = percent_change_col.max()
        
        if np.isnan(min_val) or np.isnan(max_val):
            continue
        
        normalized_col = (percent_change_col - min_val) / (max_val - min_val)
        relative_col = percent_change_col/100
        impact_col_name = f"Impact in {cap_col} (TotalNPK {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"
        relative_col_name = f"Relative Change in {cap_col} (TotalNPK {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"
        new_data_1[impact_col_name] = normalized_col
        new_data_1[relative_col_name] = relative_col


In [109]:
new_data_1.columns

Index(['District', 'Initial Stability', 'Rice_Production (TotalNPK +10%)',
       'Rice_Production (TotalNPK -10%)',
       'Normalized Rice_Production (TotalNPK +10%)',
       'Normalized Rice_Production (TotalNPK -10%)',
       'Percent Change in Rice_Production (TotalNPK +10%)',
       'Percent Change in Rice_Production (TotalNPK -10%)',
       'Maize_Production (TotalNPK +10%)', 'Maize_Production (TotalNPK -10%)',
       'Normalized Maize_Production (TotalNPK +10%)',
       'Normalized Maize_Production (TotalNPK -10%)',
       'Percent Change in Maize_Production (TotalNPK +10%)',
       'Percent Change in Maize_Production (TotalNPK -10%)',
       'Rice_Production (TotalNPK +20%)', 'Rice_Production (TotalNPK -20%)',
       'Normalized Rice_Production (TotalNPK +20%)',
       'Normalized Rice_Production (TotalNPK -20%)',
       'Percent Change in Rice_Production (TotalNPK +20%)',
       'Percent Change in Rice_Production (TotalNPK -20%)',
       'Maize_Production (TotalNPK +20%)', 'M

In [110]:
new_data_1['Relative Change in Rice_Production (TotalNPK +10%)']

0       20.606587
1       25.867112
2        1.887984
3       21.568492
4       -0.477640
5       80.322224
6       30.514845
7       -0.603438
8        3.165120
9        0.624944
10      -0.535111
11      -0.769423
12       0.259314
13       0.334476
14      -0.342424
15      -0.532485
16       0.616455
17       1.523555
18     863.504742
19    2589.448627
20       2.137243
21       9.040233
22       0.377413
23      -0.648289
24      -0.200591
25      36.412149
26      17.562062
27       0.050189
28      -0.395109
29      -0.206463
30      -0.200602
Name: Relative Change in Rice_Production (TotalNPK +10%), dtype: float64

In [111]:
newdf = pd.DataFrame()
newdf = pd.concat([newdf, new_data_1['District']], axis=1) 
newdf = pd.concat([newdf, new_data_1['Percent Change in Rice_Production (TotalNPK +10%)']], axis=1) 
newdf = pd.concat([newdf, new_data_1['Relative Change in Rice_Production (TotalNPK +10%)']], axis=1) 
newdf = pd.concat([newdf, new_data_1['Impact in Rice_Production (TotalNPK +10%)']], axis=1) 
newdf = pd.concat([newdf, new_data_1['Percent Change in Maize_Production (TotalNPK +10%)']], axis=1) 
newdf = pd.concat([newdf, new_data_1['Relative Change in Maize_Production (TotalNPK +10%)']], axis=1) 
newdf = pd.concat([newdf, new_data_1['Impact in Maize_Production (TotalNPK +10%)']], axis=1) 
newdf.columns

Index(['District', 'Percent Change in Rice_Production (TotalNPK +10%)',
       'Relative Change in Rice_Production (TotalNPK +10%)',
       'Impact in Rice_Production (TotalNPK +10%)',
       'Percent Change in Maize_Production (TotalNPK +10%)',
       'Relative Change in Maize_Production (TotalNPK +10%)',
       'Impact in Maize_Production (TotalNPK +10%)'],
      dtype='object')

In [112]:
newdf.to_csv('test1.csv')

In [113]:
newdf

,District,Percent Change in Rice_Production (TotalNPK +10%),Relative Change in Rice_Production (TotalNPK +10%),Impact in Rice_Production (TotalNPK +10%),Percent Change in Maize_Production (TotalNPK +10%),Relative Change in Maize_Production (TotalNPK +10%),Impact in Maize_Production (TotalNPK +10%)
0,BENGALURU,2060.658670,20.606587,0.008253,2134.239046,21.342390,0.046395
1,BENGALURU(R),2586.711226,25.867112,0.010284,22.867551,0.228676,0.001806
2,RAMANAGARA,188.798430,1.887984,0.001026,66.468733,0.664687,0.002727
3,CHITRADURGA,2156.849152,21.568492,0.008624,-62.650862,-0.626509,0.000000
4,DAVANAGERE,-47.763967,-0.477640,0.000113,-51.056661,-0.510567,0.000245
5,KOLAR,8032.222426,80.322224,0.031307,3368.175026,33.681750,0.072454
6,CHIKKABALLAPURA,3051.484542,30.514845,0.012078,-24.848633,-0.248486,0.000798
7,SHIVAMOGGA,-60.343751,-0.603438,0.000064,-28.050701,-0.280507,0.000731
8,TUMAKURU,316.512025,3.165120,0.001519,61.943705,0.619437,0.002631
9,CHIKKAMAGALURU,62.494352,0.624944,0.000538,220.006555,2.200066,0.005969


In [114]:
import pandas as pd
import numpy as np

ch = [-10, 10, 20, -20]
art = pd.DataFrame() 
art = pd.concat([art, new_data_1['District']], axis=1) 

for intervention in ch:
    all_impact_cols = [f"Impact in {cap_col} (TotalNPK {'+' if intervention >= 0 else '-'}{abs(intervention)}%)" for cap_col in capability_vector]
    all_relative_change_cols = [f"Relative Change in {cap_col} (TotalNPK {'+' if intervention >= 0 else '-'}{abs(intervention)}%)" for cap_col in capability_vector]
    all_impacts = new_data_1[all_impact_cols]
    all_relative = new_data_1[all_relative_change_cols]
    art = pd.concat([art, all_impacts], axis=1) 
    art = pd.concat([art, all_relative], axis=1) 
     
# Replace inf values with 0
# art.replace([np.inf, -np.inf], 0, inplace=True)

# if art.isnull().values.any():
#     print("Warning: There are NaN values present after replacing inf.")

In [115]:
column_order = ['District','Impact in Rice_Production (TotalNPK +10%)','Impact in Rice_Production (TotalNPK -10%)','Impact in Maize_Production (TotalNPK +10%)',
    'Impact in Maize_Production (TotalNPK -10%)','Impact in Rice_Production (TotalNPK +20%)','Impact in Rice_Production (TotalNPK -20%)',
    'Impact in Maize_Production (TotalNPK +20%)','Impact in Maize_Production (TotalNPK -20%)']

art = art[column_order]
art.head()

,District,Impact in Rice_Production (TotalNPK +10%),Impact in Rice_Production (TotalNPK -10%),Impact in Maize_Production (TotalNPK +10%),Impact in Maize_Production (TotalNPK -10%),Impact in Rice_Production (TotalNPK +20%),Impact in Rice_Production (TotalNPK -20%),Impact in Maize_Production (TotalNPK +20%),Impact in Maize_Production (TotalNPK -20%)
0,BENGALURU,0.008253,0.008530,0.046395,0.045771,0.008146,0.008715,0.046628,0.045339
1,BENGALURU(R),0.010284,0.010712,0.001806,0.001767,0.010119,0.010999,0.001820,0.001741
2,RAMANAGARA,0.001026,0.001117,0.002727,0.002530,0.000991,0.001178,0.002800,0.002394
3,CHITRADURGA,0.008624,0.008783,0.000000,0.000000,0.008563,0.008890,0.000000,0.000000
4,DAVANAGERE,0.000113,0.000103,0.000245,0.000250,0.000116,0.000097,0.000243,0.000254


In [116]:
capability_vector = ["Rice_Production", "Maize_Production"]

for intervention in ch:
    all_percent_change_cols = [f"Impact in {cap_col} (TotalNPK {'+' if intervention >= 0 else '-'}{abs(intervention)}%)" for cap_col in capability_vector]
    print(all_percent_change_cols)

['Impact in Rice_Production (TotalNPK -10%)', 'Impact in Maize_Production (TotalNPK -10%)']
['Impact in Rice_Production (TotalNPK +10%)', 'Impact in Maize_Production (TotalNPK +10%)']
['Impact in Rice_Production (TotalNPK +20%)', 'Impact in Maize_Production (TotalNPK +20%)']
['Impact in Rice_Production (TotalNPK -20%)', 'Impact in Maize_Production (TotalNPK -20%)']


In [117]:
capability_vector = ["Rice_Production", "Maize_Production"]

dissonance = pd.DataFrame(columns=capability_vector)

# List to store dissonance for each intervention
dis_list = []

# List to store mean for each intervention
mean_list = []

for intervention in ch:
    all_percent_change_cols = [f"Impact in {cap_col} (TotalNPK {'+' if intervention >= 0 else '-'}{abs(intervention)}%)" for cap_col in capability_vector]
    
    # Extracting columns for current intervention
    intervention_df = art.loc[:, all_percent_change_cols] 
      
    # Calculating maximum and minimum values for each row
    max_values = intervention_df.max(axis=1)
    min_values = intervention_df.min(axis=1)
    
    # Calculating differences for each row
    diff = max_values - min_values
    
    # Appending differences to the dissonance list
    dis_list.append(diff)
    
    # Calculating mean for each intervention
    mean = intervention_df.mean(axis=1)
    
    # Appending mean to the mean list
    mean_list.append(mean)

# Creating a DataFrame from the list of differences (dissonance)
dissonance = pd.DataFrame(dis_list).T
dissonance.columns = [f'Dis_{intervention}' for intervention in ch]

# Creating a DataFrame from the list of means
mean_df = pd.DataFrame(mean_list).T
mean_df.columns = [f'Mean_{intervention}' for intervention in ch]

print("Dissonance:")
print(dissonance)

Dissonance:
     Dis_-10    Dis_10    Dis_20   Dis_-20
0   0.037241  0.038143  0.038482  0.036624
1   0.008945  0.008477  0.008298  0.009258
2   0.001413  0.001701  0.001810  0.001215
3   0.008783  0.008624  0.008563  0.008890
4   0.000147  0.000132  0.000127  0.000157
5   0.038979  0.041147  0.041969  0.037507
6   0.011610  0.011280  0.011153  0.011831
7   0.000675  0.000667  0.000663  0.000681
8   0.001076  0.001112  0.001126  0.001051
9   0.005428  0.005431  0.005432  0.005425
10  0.000000  0.000000  0.000000  0.000000
11  0.240573  0.260249  0.267596  0.226975
12  0.000076  0.000066  0.000063  0.000083
13  0.012448  0.012523  0.012552  0.012396
14  0.018032  0.018076  0.018092  0.018002
15  0.001882  0.001876  0.001874  0.001886
16  0.000403  0.000384  0.000377  0.000415
17  0.000287  0.000261  0.000252  0.000304
18  0.335823  0.333002  0.331919  0.337707
19  0.998717  0.998720  0.998721  0.998715
20  0.000535  0.000559  0.000569  0.000519
21  0.003728  0.003596  0.003546  0.003816

In [118]:
dissonance['Dis_10']

0     0.038143
1     0.008477
2     0.001701
3     0.008624
4     0.000132
5     0.041147
6     0.011280
7     0.000667
8     0.001112
9     0.005431
10    0.000000
11    0.260249
12    0.000066
13    0.012523
14    0.018076
15    0.001876
16    0.000384
17    0.000261
18    0.333002
19    0.998720
20    0.000559
21    0.003596
22    0.000272
23    0.002727
24    0.002604
25    0.036551
26    0.014559
27    0.209086
28    0.999855
29    0.001360
30    0.002604
Name: Dis_10, dtype: float64

In [119]:
print("\nMean:")
print(mean_df)


Mean:
    Mean_-10   Mean_10   Mean_20  Mean_-20
0   0.027150  0.027324  0.027387  0.027027
1   0.006240  0.006045  0.005970  0.006370
2   0.001823  0.001876  0.001896  0.001786
3   0.004392  0.004312  0.004281  0.004445
4   0.000177  0.000179  0.000180  0.000175
5   0.051927  0.051881  0.051857  0.051946
6   0.006593  0.006438  0.006379  0.006696
7   0.000394  0.000397  0.000399  0.000391
8   0.002076  0.002075  0.002075  0.002077
9   0.003246  0.003254  0.003257  0.003240
10  0.000089  0.000090  0.000091  0.000089
11  0.120286  0.130124  0.133798  0.113487
12  0.000428  0.000430  0.000431  0.000427
13  0.006649  0.006688  0.006702  0.006622
14  0.009174  0.009203  0.009214  0.009154
15  0.001025  0.001029  0.001031  0.001021
16  0.000346  0.000343  0.000342  0.000347
17  0.001011  0.001016  0.001018  0.001008
18  0.168578  0.167167  0.166626  0.169521
19  0.500642  0.500640  0.500639  0.500643
20  0.001403  0.001402  0.001401  0.001403
21  0.002048  0.001989  0.001967  0.002087
22  

In [120]:
# Create a new DataFrame for Sustainability scores
sustainability_scores = pd.DataFrame(index=res1_df.index)
sustainability_scores['District'] = df['Abbreviation']
for intervention in ch:
    # Extracting mean and dissonance columns for the current intervention
    mean_col = f'Mean_{intervention}'
    dissonance_col = f'Dis_{intervention}'
    stability_col = f'New Stability(NPK {"+" if intervention >= 0 else "-"} {abs(intervention)}%)'  # Adjusted stability column name
    
    # Calculate Sustainability score for the current intervention
    sustainability_score_col = (mean_df[mean_col] * res1_df[stability_col]) / dissonance[dissonance_col]
    
    # Add the calculated Sustainability score to the new DataFrame
    sustainability_scores[f'Sustainability_{intervention}'] = sustainability_score_col

In [49]:
sustainability_scores = pd.read_csv("Sustainability_Scores.csv")

In [122]:
print(sustainability_scores)
sustainability_scores.to_csv("Sustainability_Scores.csv")

   District  Sustainability_-10  Sustainability_10  Sustainability_20  \
0       BLR            0.694827           0.673999           0.665258   
1    BLR(R)            0.685272           0.696400           0.700544   
2       RGA            1.119198           0.921876           0.859023   
3       CDA            0.421434           0.403092           0.393922   
4       DVG            0.936871           0.984080           0.995993   
5       KLR            1.330847           1.257095           1.230708   
6       CKA            0.566647           0.568283           0.568794   
7       SMG            0.569295           0.577481           0.580160   
8       TKR            1.913320           1.843123           1.816806   
9       CMG            0.531262           0.516368           0.508786   
10      DKA                 inf                inf                inf   
11      UPI            0.411685           0.391177           0.380924   
12      HSN            5.014296           5.643782 

In [51]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=4, cols=1, subplot_titles=['NPK Fertilizers -10%', 'NPK Fertilizers +10%', 'NPK Fertilizers +20%', 'NPK Fertilizers -20%'])

for i, col in enumerate(['Sustainability_-10', 'Sustainability_10', 'Sustainability_20', 'Sustainability_-20'], start=1):
    fig.add_trace(go.Scatter(x=sustainability_scores['District'], y=sustainability_scores[col], mode='lines+markers', name=col, showlegend=False), row=i, col=1)

fig.update_xaxes(title_text="District", row=4, col=1)
fig.update_yaxes(title_text="SI Score", col=1)

fig.update_layout(height=800, width=1000, title_text="District-wise Sustainable Intervention Scores in SDG 2 Capability Indicators (NPK Fertilizer Perturbations)")

fig.show()


In [52]:
normalized_data_1 = res1_df.copy()
min_value = normalized_data_1.min().min()
max_value = normalized_data_1.max().max()
normalized_data_1 = (normalized_data_1 - min_value) / (max_value - min_value)
normalized_data_2 = 1 - (normalized_data_1 - min_value) / (max_value - min_value)

In [53]:
normalized_data_1

,New Stability(NPK + 10%),New Stability(NPK - 10%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,0.940864,0.953059,0.934767,0.959157
1,0.976666,0.982351,0.973823,0.985194
2,0.835654,0.866978,0.819992,0.882640
3,0.806185,0.842867,0.787844,0.861208
4,0.727833,0.778761,0.702369,0.804225
5,0.997024,0.999008,0.996032,1.000000
6,0.995630,0.997867,0.994511,0.998986
7,0.968719,0.975850,0.965154,0.979415
8,0.987907,0.991549,0.986086,0.993370
9,0.861877,0.888433,0.848599,0.901712


In [54]:
normalized_data_2

,New Stability(NPK + 10%),New Stability(NPK - 10%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,0.059136,0.046941,0.065233,0.040843
1,0.023334,0.017649,0.026177,0.014806
2,0.164346,0.133022,0.180008,0.117360
3,0.193815,0.157133,0.212156,0.138792
4,0.272167,0.221239,0.297631,0.195775
5,0.002976,0.000992,0.003968,0.000000
6,0.004370,0.002133,0.005489,0.001014
7,0.031281,0.024150,0.034846,0.020585
8,0.012093,0.008451,0.013914,0.006630
9,0.138123,0.111567,0.151401,0.098288


In [55]:
base_column = "TotalNPK"
capability_vector = ["Rice_Production", "Maize_Production"]
# change_percentage = 20
change_percentages = [10,20]
dim = len(capability_vector)

new_data_1,result_df,res1_df = calc_and_vis_impact_stability_2D1(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
new_data_1.head()

2.1764894064966205 17136.471837471167 Rice_Production 10
2.6936229082208722 -4743.442857301005 Maize_Production 10
2.1764894064966205 17136.471837471167 Rice_Production 20
2.6936229082208722 -4743.442857301005 Maize_Production 20
258944.86274476734
280661.6179849578


,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 10%)_Rice_Maize,New Stability(NPK - 10%)_Rice_Maize,New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.959409,72943.836709,62797.043096,0.117432,0.099835,2060.658670,1760.101987,64323.742132,51766.072134,...,70602.577131,45487.237135,0.113372,0.069814,2352.329876,1479.966556,0.940864,0.953059,0.934767,0.959157
1,BENGALURU(R),0.983965,63245.182265,54861.780369,0.100612,0.086073,2586.711226,2230.576906,52320.689091,41945.392373,...,57508.337450,36757.744014,0.090663,0.054675,35.049990,-13.679769,0.976666,0.982351,0.973823,0.985194
2,RAMANAGARA,0.868345,35025.473567,31772.927798,0.051671,0.046030,188.798430,161.979946,17395.982550,13370.632476,...,19408.657587,11357.957439,0.024586,0.010624,85.728781,8.688588,0.835654,0.866978,0.819992,0.882640
3,CHITRADURGA,0.562713,103047.732286,87427.503114,0.169641,0.142551,2156.849152,1814.750397,101580.318025,82248.725137,...,111246.114469,72582.928694,0.183860,0.116806,-59.096934,-73.312647,0.806185,0.842867,0.787844,0.861208
4,DAVANAGERE,0.072591,255697.993301,212323.171216,0.434382,0.359157,-47.763967,-56.624923,290500.254316,236819.582103,...,317340.590423,209979.245996,0.541288,0.355092,-46.534614,-64.622800,0.727833,0.778761,0.702369,0.804225


In [56]:
new_data_1

,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 10%)_Rice_Maize,New Stability(NPK - 10%)_Rice_Maize,New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.959409,72943.836709,62797.043096,0.117432,0.099835,2060.658670,1760.101987,64323.742132,51766.072134,...,70602.577131,45487.237135,0.113372,0.069814,2352.329876,1479.966556,0.940864,0.953059,0.934767,0.959157
1,BENGALURU(R),0.983965,63245.182265,54861.780369,0.100612,0.086073,2586.711226,2230.576906,52320.689091,41945.392373,...,57508.337450,36757.744014,0.090663,0.054675,35.049990,-13.679769,0.976666,0.982351,0.973823,0.985194
2,RAMANAGARA,0.868345,35025.473567,31772.927798,0.051671,0.046030,188.798430,161.979946,17395.982550,13370.632476,...,19408.657587,11357.957439,0.024586,0.010624,85.728781,8.688588,0.835654,0.866978,0.819992,0.882640
3,CHITRADURGA,0.562713,103047.732286,87427.503114,0.169641,0.142551,2156.849152,1814.750397,101580.318025,82248.725137,...,111246.114469,72582.928694,0.183860,0.116806,-59.096934,-73.312647,0.806185,0.842867,0.787844,0.861208
4,DAVANAGERE,0.072591,255697.993301,212323.171216,0.434382,0.359157,-47.763967,-56.624923,290500.254316,236819.582103,...,317340.590423,209979.245996,0.541288,0.355092,-46.534614,-64.622800,0.727833,0.778761,0.702369,0.804225
5,KOLAR,0.878180,70262.401761,60603.141775,0.112782,0.096030,8032.222426,6914.252520,61005.198709,49050.900243,...,66982.347943,43073.751009,0.107093,0.065629,3707.978848,2348.763559,0.997024,0.999008,0.996032,1.000000
6,CHIKKABALLAPURA,0.883837,85027.052947,72683.310927,0.138388,0.116980,3051.484542,2593.970012,79277.928432,64001.315470,...,86916.234913,56363.008989,0.141664,0.088676,-17.607915,-46.570789,0.995630,0.997867,0.994511,0.998986
7,SHIVAMOGGA,0.630729,152666.249531,128024.471769,0.255694,0.212958,-60.343751,-66.744645,162988.186275,132491.526433,...,178236.516196,117243.196512,0.300041,0.194260,-21.319497,-48.244311,0.968719,0.975850,0.965154,0.979415
8,TUMAKURU,0.865884,108468.061504,91862.317929,0.179042,0.150242,316.512025,252.746786,108288.516516,87737.251175,...,118564.149186,77461.618505,0.196551,0.125267,77.310745,15.842583,0.987907,0.991549,0.986086,0.993370
9,CHIKKAMAGALURU,0.698987,179609.882490,150069.262371,0.302423,0.251191,62.494352,35.768741,196333.621705,159774.155421,...,214613.354847,141494.422279,0.363129,0.236319,249.800914,130.623478,0.861877,0.888433,0.848599,0.901712


In [57]:
import statsmodels.api as sm
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def calc_and_vis_impact_stability_2D(G, existing_data, adjacency_file, base_column, capability_vector, change_percentages, dim):
    min_value = float('inf')
    max_value = float('-inf')
    
    change_min_value = float('inf')
    change_max_value = float('-inf')
    
    initial_stability_column = calculate_and_return_initial_stability(df, capability_vector, G)
    existing_data = pd.concat([existing_data, initial_stability_column], axis=1)

    col_pass_plus = []
    col_pass_minus = []
    result_df = pd.DataFrame()
    
    for intervention in change_percentages:
        for cap_col in capability_vector:
            X = sm.add_constant(df[base_column])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()
            m, c = model.params[base_column], model.params['const']

            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            change_vector_increase = m * base_column_increase + c
            change_vector_decrease = m * base_column_decrease + c
            
            change_min_value = min(change_min_value, change_vector_increase.min(), change_vector_decrease.min())
            change_max_value = max(change_max_value, change_vector_increase.max(), change_vector_decrease.max())
            
    
    result_df = pd.DataFrame()
    res1_df = pd.DataFrame()
    
    for intervention in change_percentages:
        col_pass_plus = []
        col_pass_minus = []
        for cap_col in capability_vector:
            # Calculate Impact Score
            X = sm.add_constant(df[base_column])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()

            m, c = model.params[base_column], model.params['const']
            # print(m,c)
            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            new_vector_increase = m * base_column_increase + c
            new_vector_decrease = m * base_column_decrease + c
            
            normalized_new_increase_vector = (new_vector_increase - change_min_value) / (change_max_value - change_min_value)
            normalized_new_decrease_vector = (new_vector_decrease - change_min_value) / (change_max_value - change_min_value)
            
            percent_change_inc = (new_vector_increase - (y) )*100/(y)
            percent_change_dec = (new_vector_decrease - (y) )*100/(y)

            result_df = pd.concat([result_df,
                pd.DataFrame({
                    f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
                    f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
                    f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector,
                    f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector,
                    f'Percent Change in {cap_col} ({base_column} +{intervention}%)': percent_change_inc,
                    f'Percent Change in {cap_col} ({base_column} -{intervention}%)': percent_change_dec,
                    
                })
            ], axis=1)

            columns_to_pass = f'{cap_col} ({base_column} +{intervention}%)'
            col_pass_plus.append(columns_to_pass)
            
            # Calculate Stability for the opposite intervention and add to DataFrame
            opposite_intervention = -intervention
            columns_to_pass_opposite = f'{cap_col} ({base_column} -{intervention}%)'
            col_pass_minus.append(columns_to_pass_opposite)
        
        for column in result_df.columns:
            existing_data[column] = result_df[column]

        # print(col_pass_minus)
        # print(col_pass_plus)
        
        stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

        normalized_stability_vector = (stability_column - stability_column.min()) / (stability_column.max() - stability_column.min())

        capability_first_words = [col.split('_')[0] for col in capability_vector]
        stability_column_name_modified = stability_column_name + "_" + "_".join(capability_first_words)
        existing_data[stability_column_name_modified] = normalized_stability_vector
        
        stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)

        # Normalize stability values for the opposite intervention
        normalized_stability_vector_opposite = (stability_column_opposite - stability_column_opposite.min()) / (stability_column_opposite.max() - stability_column_opposite.min())    
        stability_column_name_opposite_modified = stability_column_name_opposite + "_" + "_".join(capability_first_words)
        existing_data[stability_column_name_opposite_modified] = normalized_stability_vector_opposite
        
        res1_df = pd.concat([res1_df,
            pd.DataFrame({
            stability_column_name_modified: stability_column,
            stability_column_name_opposite_modified: stability_column_opposite,
            })
        ], axis=1)
        
        normalized_data_1 = res1_df.copy()
        min_value = normalized_data_1.min().min()
        max_value = normalized_data_1.max().max()
        normalized_data_1 = (normalized_data_1 - min_value) / (max_value - min_value)
        
        # for col in normalized_data_1.columns:
        for column in normalized_data_1.columns:
            existing_data[column] = normalized_data_1[column]
        
        # Visualize with Plotly for +intervention
        fig_increase = go.Figure()
        color_palette = ['red', 'blue', 'green', 'orange', 'purple'] 

        for i, cap_col in enumerate(capability_vector):
            symbol = 'star' if i == 0 else 'triangle-up' 
            scatter_increase = go.Scatter(
                x=existing_data[stability_column_name_modified],
                y=existing_data[f"Percent Change in {cap_col} ({base_column} +{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=7, symbol=symbol, color=color_palette[i]),  
                name=f'{cap_col} +{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Percent Change in: %{y:.2f}' + '<br>Stability: %{x:.2f}'
            )
            fig_increase.add_trace(scatter_increase)
        
        # Add average lines
        # print("aefawefaw")
        avg_stability_increase = existing_data[stability_column_name_modified].mean()
        avg_impact_increase = existing_data[f"Percent Change in {capability_vector[0]} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"].mean()
        y_range_1 = max(abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].min()), 
                    abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} -{abs(intervention)}%)"].min()), 
                    abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].max()), 
                    abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} -{abs(intervention)}%)"].max()))
        y_axis_1 = max(y_range_1,1)


        fig_increase.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_increase,
                x1=avg_stability_increase,
                y0=0,
                y1=y_axis_1,
                line=dict(color="black", dash="dash"),
                name="Average Stability"
            )
        )
        fig_increase.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_increase,
                y1=avg_impact_increase,
                line=dict(color="black", dash="dash"),
                name="Average Impact"
            )
        )

        # Update layout for +intervention plot
        fig_increase.update_layout(
            title=f'Percent Change vs Stability (+{intervention}%)',
            xaxis=dict(title=f'Stability ({base_column} +{intervention}%)'),
            yaxis=dict(title='Percent Change'),
            hovermode='closest'
        )

        # Visualize with Plotly for -intervention
        fig_decrease = go.Figure()

        for i, cap_col in enumerate(capability_vector):
            # Scatter plot for -intervention
            symbol = 'star' if i == 0 else 'triangle-up'
            
            scatter_decrease = go.Scatter(
                x=existing_data[stability_column_name_opposite_modified],
                y=existing_data[f"Percent Change in {cap_col} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=7,symbol=symbol, color=color_palette[i]),
                name=f'{cap_col} -{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Percent Change: %{y:.2f}' + '<br>Stability: %{x:.2f}'
            )
            
            fig_decrease.add_trace(scatter_decrease)

        # Add average lines for -intervention
        avg_stability_decrease = existing_data[stability_column_name_opposite_modified].mean()
        avg_impact_decrease = existing_data[f"Percent Change in {capability_vector[0]} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"].mean()

        fig_decrease.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_decrease,
                x1=avg_stability_decrease,
                y0=0,
                y1=y_axis_1,
                line=dict(color="black", dash="dash"),
                name="Average Stability"
            )
        )

        fig_decrease.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_decrease,
                y1=avg_impact_decrease,
                line=dict(color="black", dash="dash"),
                name="Average Impact"
            )
        )

        # Update layout for -intervention plot
        fig_decrease.update_layout(
            title=f'Percent Change vs Stability (-{intervention}%)',
            xaxis=dict(title=f'Stability ({base_column} -{intervention}%)'),
            yaxis=dict(title='Percent Change'),
            hovermode='closest'
        )


        fig_increase.show()
        fig_decrease.show()

    return existing_data, result_df

base_column = "TotalNPK"
capability_vector = ["Rice_Production", "Maize_Production"]
# change_percentage = 20
change_percentages = [10,20]
dim = len(capability_vector)

new_data_2,result_df = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
new_data_2.head()

,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Percent Change in Rice_Production (TotalNPK +20%),Percent Change in Rice_Production (TotalNPK -20%),Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.959409,72943.836709,62797.043096,0.117432,0.099835,2060.658670,1760.101987,61841.718503,49106.143430,...,2210.937012,1609.823646,68209.506039,42738.355894,0.109221,0.065047,2269.208268,1384.486137,0.934767,0.959157
1,BENGALURU(R),0.983965,63245.182265,54861.780369,0.100612,0.086073,2586.711226,2230.576906,49668.617670,39146.333658,...,2764.778386,2052.509746,54929.759676,33885.191652,0.086190,0.049693,28.994575,-20.425542,0.973823,0.985194
2,RAMANAGARA,0.868345,35025.473567,31772.927798,0.051671,0.046030,188.798430,161.979946,14249.131244,10166.753855,...,202.207672,148.570703,16290.319939,8125.565160,0.019178,0.005018,55.888229,-22.243396,0.819992,0.882640
3,CHITRADURGA,0.562713,103047.732286,87427.503114,0.169641,0.142551,2156.849152,1814.750397,99626.109978,80020.645546,...,2327.898530,1643.701019,109428.842194,70217.913330,0.180708,0.112705,-59.765110,-74.182218,0.787844,0.861208
4,DAVANAGERE,0.072591,255697.993301,212323.171216,0.434382,0.359157,-47.763967,-56.624923,291222.483485,236781.314779,...,-43.333489,-61.055401,318443.067838,209560.730426,0.543200,0.354366,-46.348869,-64.693312,0.702369,0.804225


In [58]:
# new_data_2

In [59]:
df

,District,TotalNPK,Rice_Production,Jowar_Production,Maize_Production,Jowar_Yield,Normalized_Maize_Production,Initial Stability_Maize_Production,Normalized_Rice_Production,Initial Stability_Rice_Production,Normalized_Jowar_Production,Initial Stability_Jowar_Production,Initial Stability,Abbreviation
0,BENGALURU,23310.0,3376.0,851,2879,1539.0,0.004851,0.960176,0.006686,0.992148,0.003267,0.996733,0.959409,BLR
1,BENGALURU(R),19259.0,2354.0,0,42583,0.0,0.071744,0.991418,0.004609,0.986455,0.000000,0.997311,0.983965,BLR(R)
2,RAMANAGARA,7472.0,12128.0,0,10450,0.0,0.017606,0.923391,0.024465,0.892930,0.000000,0.992597,0.868345,RGA
3,CHITRADURGA,35884.0,4566.0,13301,271975,984.0,0.458222,0.855101,0.009103,0.587418,0.051060,0.972188,0.562713,CDA
4,DAVANAGERE,99644.0,489505.0,23072,593544,2167.0,1.000000,0.331747,0.994237,0.356941,0.088568,0.993075,0.072591,DVG
5,KOLAR,22190.0,864.0,0,1759,0.0,0.002964,0.878226,0.001583,0.996624,0.000000,1.000000,0.878180,KLR
6,CHIKKABALLAPURA,28357.0,2698.0,0,105491,0.0,0.177731,0.884725,0.005308,0.985667,0.000000,0.996607,0.883837,CKA
7,SHIVAMOGGA,56609.0,384974.0,756,226532,2242.0,0.381660,0.986495,0.781886,0.630976,0.002902,0.950160,0.630729,SMG
8,TUMAKURU,38148.0,26042.0,2652,66868,1346.0,0.112659,0.909238,0.052731,0.901262,0.010180,0.996715,0.865884,TKR
9,CHIKKAMAGALURU,67863.0,110533.0,9080,61353,897.0,0.103367,0.744033,0.224371,0.841603,0.034856,0.987498,0.698987,CMG


In [60]:
import statsmodels.api as sm
import pandas as pd
import plotly.graph_objects as go

def calc_and_vis_impact_stability_2D(G, existing_data, adjacency_file, base_column, capability_vector, change_percentages, dim):
    # Calculate min and max values for changes made by each column in capability_vector
    min_value = float('inf')
    max_value = float('-inf')
    
    change_min_value = float('inf')
    change_max_value = float('-inf')
    
    initial_stability_column = calculate_and_return_initial_stability(df, capability_vector, G)
    existing_data = pd.concat([existing_data, initial_stability_column], axis=1)

    col_pass_plus = []
    col_pass_minus = []
    result_df = pd.DataFrame()

    # Create fixed columns
    fixed_columns = {
        'District': [],
        'Initial_Stability': [],
        'Base_Column': [],
        'Intervention_Percentage': [],
        'Capability_Vector': [],
        'New_Values': [],
        'Normalized_New_Values': [],
        'Percent_Change': []
    }
    
    for intervention in change_percentages:
        for cap_col in capability_vector:
            X = sm.add_constant(df[base_column])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()
            print(cap_col)            
            m, c = model.params[base_column], model.params['const']
            print(m,c)
            
            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            change_vector_increase = m * base_column_increase + c
            change_vector_decrease = m * base_column_decrease + c
            
            change_min_value = min(change_min_value, change_vector_increase.min(), change_vector_decrease.min())
            change_max_value = max(change_max_value, change_vector_increase.max(), change_vector_decrease.max())

    for intervention in change_percentages:
        for cap_col in capability_vector:
            X = sm.add_constant(df[base_column])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()

            m, c = model.params[base_column], model.params['const']
            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            new_vector_increase = m * base_column_increase + c
            new_vector_decrease = m * base_column_decrease + c
            
            normalized_new_increase_vector = (new_vector_increase - change_min_value) / (change_max_value - change_min_value)
            normalized_new_decrease_vector = (new_vector_decrease - change_min_value) / (change_max_value - change_min_value)
            
            percent_change_inc = (new_vector_increase - (y) )*100/(y)
            percent_change_dec = (new_vector_decrease - (y) )*100/(y)

            fixed_columns['District'].extend(existing_data['District'].tolist())
            fixed_columns['Initial_Stability'].extend(existing_data['Initial Stability'].tolist())
            fixed_columns['Base_Column'].extend([base_column] * len(existing_data))
            fixed_columns['Intervention_Percentage'].extend([intervention] * len(existing_data))
            fixed_columns['Capability_Vector'].extend([cap_col] * len(existing_data))
            fixed_columns['New_Values'].extend(new_vector_increase.tolist())
            fixed_columns['Normalized_New_Values'].extend(normalized_new_increase_vector.tolist())
            fixed_columns['Percent_Change'].extend(percent_change_inc.tolist())

            fixed_columns['District'].extend(existing_data['District'].tolist())
            fixed_columns['Initial_Stability'].extend(existing_data['Initial Stability'].tolist())
            fixed_columns['Base_Column'].extend([base_column] * len(existing_data))
            fixed_columns['Intervention_Percentage'].extend([-intervention] * len(existing_data))
            fixed_columns['Capability_Vector'].extend([cap_col] * len(existing_data))
            fixed_columns['New_Values'].extend(new_vector_decrease.tolist())
            fixed_columns['Normalized_New_Values'].extend(normalized_new_decrease_vector.tolist())
            fixed_columns['Percent_Change'].extend(percent_change_dec.tolist())
            
    # Create DataFrame from fixed columns
    result_df = pd.DataFrame(fixed_columns)

    return result_df

# Usage
base_column = "TotalNPK"
capability_vector = ["Rice_Production", "Maize_Production"]
adjacency_file = "Karnataka_District_Adjacency_File - Copy.xlsx"
change_percentages = [10, 20]
dim = len(capability_vector)

result_df_2 = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
result_df_2.head()


Rice_Production
2.1764894064966205 17136.471837471167
Maize_Production
2.731783584846858 -8203.94439643982
Rice_Production
2.1764894064966205 17136.471837471167
Maize_Production
2.731783584846858 -8203.94439643982


,District,Initial_Stability,Base_Column,Intervention_Percentage,Capability_Vector,New_Values,Normalized_New_Values,Percent_Change
0,BENGALURU,0.959409,TotalNPK,10,Rice_Production,72943.836709,0.117432,2060.658670
1,BENGALURU(R),0.983965,TotalNPK,10,Rice_Production,63245.182265,0.100612,2586.711226
2,RAMANAGARA,0.868345,TotalNPK,10,Rice_Production,35025.473567,0.051671,188.798430
3,CHITRADURGA,0.562713,TotalNPK,10,Rice_Production,103047.732286,0.169641,2156.849152
4,DAVANAGERE,0.072591,TotalNPK,10,Rice_Production,255697.993301,0.434382,-47.763967


In [61]:
result_df_2

,District,Initial_Stability,Base_Column,Intervention_Percentage,Capability_Vector,New_Values,Normalized_New_Values,Percent_Change
0,BENGALURU,0.959409,TotalNPK,10,Rice_Production,72943.836709,0.117432,2060.658670
1,BENGALURU(R),0.983965,TotalNPK,10,Rice_Production,63245.182265,0.100612,2586.711226
2,RAMANAGARA,0.868345,TotalNPK,10,Rice_Production,35025.473567,0.051671,188.798430
3,CHITRADURGA,0.562713,TotalNPK,10,Rice_Production,103047.732286,0.169641,2156.849152
4,DAVANAGERE,0.072591,TotalNPK,10,Rice_Production,255697.993301,0.434382,-47.763967
...,...,...,...,...,...,...,...,...
243,KALABURAGI,0.826587,TotalNPK,-20,Maize_Production,109806.921042,0.181364,651.021962
244,YADGIRI,0.823837,TotalNPK,-20,Maize_Production,191520.031632,0.323078,7064.984348
245,RAICHUR,0.228765,TotalNPK,-20,Maize_Production,248004.574459,0.421039,34202.154144
246,KOPPAL,0.828223,TotalNPK,-20,Maize_Production,152538.572590,0.255473,-19.822881


In [62]:
Stability_column_values = pd.concat([
                               new_data_1['New Stability(NPK + 10%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK - 10%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK + 10%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK - 10%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK + 20%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK - 20%)_Rice_Maize'],
                               new_data_1['New Stability(NPK + 20%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK - 20%)_Rice_Maize']], axis=0)

Stability_column_values.reset_index(drop=True, inplace=True)

result_df_2['Stability'] = Stability_column_values

In [63]:
result_df_2

,District,Initial_Stability,Base_Column,Intervention_Percentage,Capability_Vector,New_Values,Normalized_New_Values,Percent_Change,Stability
0,BENGALURU,0.959409,TotalNPK,10,Rice_Production,72943.836709,0.117432,2060.658670,0.940864
1,BENGALURU(R),0.983965,TotalNPK,10,Rice_Production,63245.182265,0.100612,2586.711226,0.976666
2,RAMANAGARA,0.868345,TotalNPK,10,Rice_Production,35025.473567,0.051671,188.798430,0.835654
3,CHITRADURGA,0.562713,TotalNPK,10,Rice_Production,103047.732286,0.169641,2156.849152,0.806185
4,DAVANAGERE,0.072591,TotalNPK,10,Rice_Production,255697.993301,0.434382,-47.763967,0.727833
...,...,...,...,...,...,...,...,...,...
243,KALABURAGI,0.826587,TotalNPK,-20,Maize_Production,109806.921042,0.181364,651.021962,0.994402
244,YADGIRI,0.823837,TotalNPK,-20,Maize_Production,191520.031632,0.323078,7064.984348,0.928301
245,RAICHUR,0.228765,TotalNPK,-20,Maize_Production,248004.574459,0.421039,34202.154144,0.791588
246,KOPPAL,0.828223,TotalNPK,-20,Maize_Production,152538.572590,0.255473,-19.822881,0.989195


In [64]:
result_df

,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),Normalized Maize_Production (TotalNPK +10%),Normalized Maize_Production (TotalNPK -10%),...,Normalized Rice_Production (TotalNPK +20%),Normalized Rice_Production (TotalNPK -20%),Percent Change in Rice_Production (TotalNPK +20%),Percent Change in Rice_Production (TotalNPK -20%),Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%)
0,72943.836709,62797.043096,0.117432,0.099835,2060.658670,1760.101987,61841.718503,49106.143430,0.098178,0.076091,...,0.126231,0.091036,2210.937012,1609.823646,68209.506039,42738.355894,0.109221,0.065047,2.269208e+03,1.384486e+03
1,63245.182265,54861.780369,0.100612,0.086073,2586.711226,2230.576906,49668.617670,39146.333658,0.077066,0.058817,...,0.107882,0.078803,2764.778386,2052.509746,54929.759676,33885.191652,0.086190,0.049693,2.899457e+01,-2.042554e+01
2,35025.473567,31772.927798,0.051671,0.046030,188.798430,161.979946,14249.131244,10166.753855,0.015638,0.008558,...,0.054491,0.043209,202.207672,148.570703,16290.319939,8125.565160,0.019178,0.005018,5.588823e+01,-2.224340e+01
3,103047.732286,87427.503114,0.169641,0.142551,2156.849152,1814.750397,99626.109978,80020.645546,0.163707,0.129706,...,0.183186,0.129006,2327.898530,1643.701019,109428.842194,70217.913330,0.180708,0.112705,-5.976511e+01,-7.418222e+01
4,255697.993301,212323.171216,0.434382,0.359157,-47.763967,-56.624923,291222.483485,236781.314779,0.495992,0.401575,...,0.471994,0.321545,-43.333489,-61.055401,318443.067838,209560.730426,0.543200,0.354366,-4.634887e+01,-6.469331e+01
5,70262.401761,60603.141775,0.112782,0.096030,8032.222426,6914.252520,58476.161126,46352.505577,0.092341,0.071315,...,0.121158,0.087654,8591.207379,6355.267567,64537.988901,40290.677802,0.102854,0.060802,3.569016e+03,2.190545e+03
6,85027.052947,72683.310927,0.138388,0.116980,3051.484542,2593.970012,77007.761431,61514.724008,0.124480,0.097611,...,0.149092,0.106277,3280.241807,2365.212747,84754.280142,53768.205296,0.137915,0.084176,-1.965734e+01,-4.903053e+01
7,152666.249531,128024.471769,0.255694,0.212958,-60.343751,-66.744645,161903.946254,130975.238863,0.271715,0.218076,...,0.277063,0.191590,-57.143304,-69.945092,177368.299949,115510.885167,0.298535,0.191256,-2.170276e+01,-4.900902e+01
8,108468.061504,91862.317929,0.179042,0.150242,316.512025,252.746786,106429.343818,85586.927779,0.175506,0.139359,...,0.193441,0.135843,348.394644,220.864166,116850.551837,75165.719759,0.193579,0.121286,7.474809e+01,1.240910e+01
9,179609.882490,150069.262371,0.302423,0.251191,62.494352,35.768741,195721.787964,158644.382080,0.330365,0.266062,...,0.328039,0.225574,75.857158,22.405935,214260.490906,140105.679138,0.362517,0.233911,2.492258e+02,1.283599e+02


In [65]:
print(result_df['Rice_Production (TotalNPK +20%)'].min(), result_df['Rice_Production (TotalNPK +20%)'].max())

33193.74008284063 487237.2893424388


In [66]:
print(result_df['Maize_Production (TotalNPK +20%)'].min(), result_df['Maize_Production (TotalNPK +20%)'].max())

11950.062179126358 581835.084808067


In [67]:
capability_vector = ["Rice_Production", "Maize_Production"]
change_percentages = [10,20]
dim = len(capability_vector)

new_data_1 = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
new_data_1.head()

Rice_Production
2.1764894064966205 17136.471837471167
Maize_Production
2.731783584846858 -8203.94439643982
Rice_Production
2.1764894064966205 17136.471837471167
Maize_Production
2.731783584846858 -8203.94439643982


,District,Initial_Stability,Base_Column,Intervention_Percentage,Capability_Vector,New_Values,Normalized_New_Values,Percent_Change
0,BENGALURU,0.959409,TotalNPK,10,Rice_Production,72943.836709,0.117432,2060.658670
1,BENGALURU(R),0.983965,TotalNPK,10,Rice_Production,63245.182265,0.100612,2586.711226
2,RAMANAGARA,0.868345,TotalNPK,10,Rice_Production,35025.473567,0.051671,188.798430
3,CHITRADURGA,0.562713,TotalNPK,10,Rice_Production,103047.732286,0.169641,2156.849152
4,DAVANAGERE,0.072591,TotalNPK,10,Rice_Production,255697.993301,0.434382,-47.763967


In [68]:
base_column = "TotalNPK"
capability_vector = ["Rice_Production","Maize_Production"]
# change_percentage = 20
change_percentages= [10,20]
dim = len(capability_vector)

new_data = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
new_data.head()

Rice_Production
2.1764894064966205 17136.471837471167
Maize_Production
2.731783584846858 -8203.94439643982
Rice_Production
2.1764894064966205 17136.471837471167
Maize_Production
2.731783584846858 -8203.94439643982


,District,Initial_Stability,Base_Column,Intervention_Percentage,Capability_Vector,New_Values,Normalized_New_Values,Percent_Change
0,BENGALURU,0.959409,TotalNPK,10,Rice_Production,72943.836709,0.117432,2060.658670
1,BENGALURU(R),0.983965,TotalNPK,10,Rice_Production,63245.182265,0.100612,2586.711226
2,RAMANAGARA,0.868345,TotalNPK,10,Rice_Production,35025.473567,0.051671,188.798430
3,CHITRADURGA,0.562713,TotalNPK,10,Rice_Production,103047.732286,0.169641,2156.849152
4,DAVANAGERE,0.072591,TotalNPK,10,Rice_Production,255697.993301,0.434382,-47.763967


In [69]:
base_column = "TotalNPK"
capability_vector = [ "Rice_Production","Jowar_Yield","Maize_Production"]
change_percentage = 20
dim = 3

new_data_3d = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
new_data_3d.head()

Rice_Production
2.1764894064966205 17136.471837471167
Jowar_Yield
0.00937323234123976 375.8878302234968
Maize_Production
2.731783584846858 -8203.94439643982
Rice_Production
2.1764894064966205 17136.471837471167
Jowar_Yield
0.00937323234123976 375.8878302234968
Maize_Production
2.731783584846858 -8203.94439643982


,District,Initial_Stability,Base_Column,Intervention_Percentage,Capability_Vector,New_Values,Normalized_New_Values,Percent_Change
0,BENGALURU,0.312360,TotalNPK,10,Rice_Production,72943.836709,0.124734,2060.658670
1,BENGALURU(R),0.742141,TotalNPK,10,Rice_Production,63245.182265,0.108053,2586.711226
2,RAMANAGARA,0.600687,TotalNPK,10,Rice_Production,35025.473567,0.059516,188.798430
3,CHITRADURGA,0.547643,TotalNPK,10,Rice_Production,103047.732286,0.176511,2156.849152
4,DAVANAGERE,-0.024751,TotalNPK,10,Rice_Production,255697.993301,0.439061,-47.763967


In [70]:
# List of districts in the dataset
districts = df['District'].tolist()

# List of columns to visualize
columns_to_visualize = ['Rice_Production', 'Jowar_Yield', 'Maize_Production', 'SowingSeedsDistributed_Jowar', 'SowingSeedsDistributed_Maize']

colors = Category20[len(columns_to_visualize)]

# Create a figure for each column
for column, color in zip(columns_to_visualize, colors):
    # Create a new plot
    plot = figure(title=f'{column} vs District', x_range=districts, sizing_mode='stretch_width', toolbar_location=None, tools='')

    # Plot the data
    source = ColumnDataSource(data=dict(x=districts, y=df[column]))
    plot.vbar(x='x', top='y', width=0.8, color=color, legend_label=column, source=source)

    # Add hover tooltips
    hover = HoverTool()
    hover.tooltips = [('District', '@x'), (column, '@y')]
    plot.add_tools(hover)

    # Styling
    plot.xaxis.major_label_orientation = "vertical"
    plot.yaxis.axis_label = column
    plot.legend.location = 'top_right'
    plot.legend.click_policy = 'hide'

    # Show the plot
    show(plot)


KeyError: 'SowingSeedsDistributed_Jowar'

In [71]:
from bokeh.models import FactorRange
from bokeh.transform import factor_cmap

# Assuming df is your DataFrame

# List of crops
crops = ['Rice_Production', 'Jowar_Yield', 'Maize_Production']

# Create a new plot
plot = figure(title='Production Comparison by Crop and District', x_range=FactorRange(*districts),toolbar_location=None, tools='')

# Plot the data
source = ColumnDataSource(df)
plot.vbar_stack(crops, x='District', width=0.9, color=Category20[len(crops)], source=source, legend_label=crops)

# Add hover tooltips
hover = HoverTool()
hover.tooltips = [('District', '@District')] + [(crop, f'@{crop}') for crop in crops]
plot.add_tools(hover)

# Styling
plot.xaxis.major_label_orientation = "vertical"
plot.yaxis.axis_label = "Production"
plot.legend.location = 'top_left'
plot.legend.click_policy = 'hide'

# Show the plot
show(plot)
